<a href="https://colab.research.google.com/github/yohancsx/SmartMask/blob/main/micro_ml/BLE_Sense_SVM_Train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!apt-get -qq install xxd
!pip install pandas numpy matplotlib
!pip install scikit-learn